# ML-3 Обучение с учителем. Классификация
###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Логическая регрессия](#2)
- [2.2 Поиск параметров логической регрессии](#2-2)
- [3. Метрики классификации. Мультиклассовая классификация](#3)
- [4. Логическия регрессия. Практика](#4)
- [5. Деревья решений](#5)
- [6. Введение в ансамбли: бэггинг. Случайные лес](#6)
- [7. Деревья решений и случайный лес. Практика](#7)
- [8. Итоги](#8)

# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

# 2. Логическая регрессия <a class="anchor" id=2></a>

[к содержанию](#0)

Что вообще означает `«решить задачу классификации»`? Это значит **построить разделяющую поверхность в пространстве признаков**, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

<img src=ml3_img1.png>

Модели, которые решают задачу классификации, называются классификаторами (`classifier`).

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

<img src=ml3_img2.png>

>Но всё это работает только в том случае, когда целевой признак `y`, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак `y` является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за `y = 1` (Спам) и `y = 0` (Не спам) и обучить линейную регрессию предсказывать `0` и `1`.

Но результат будет очень плохим. Выглядеть это будет примерно так:

<img src=ml3_img3.png>

Для больших значений `x` прямая будет выдавать значения **больше 1**, а для очень маленьких — **меньше 0**. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

>Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу. 

Модель должна выдавать некоторую вероятность `P`, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как `Q = 1 - P`.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен `0.5`. То есть если вероятность `P > 0.5`, мы будем считать письмо спамом, а если `P <= 0.5` — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне `[0, 1]`. А это уже знакомая нам задача регрессии.

>Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от `-inf` до `+inf`? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

## ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

Логистическая регрессия (`Logistic Regression`) — одна из простейших моделей для решения задачи классификации. Несмотря на простоту, модель входит в топ часто используемых алгоритмов классификации в `Data Science`.

В основе логистической регрессии лежит логистическая функция (`logistic function`) `sigma` — отсюда и название модели. Однако более распространённое название этой функции — сигмόида (`sigmoid`). Записывается она следующим образом:

<img src=ml3_img4.png>

<img src=ml3_img5.png>

<img src=ml3_img6.png>

## Чего мы добились таким преобразованием?

Если мы обучим модель, то есть подберём  коэффициенты `w0, w1,... , w_m` (как их найти, обсудим чуть позже) таким образом, что для объектов класса 1 модель линейной регрессии начнёт выдавать положительное число, а для класса 0 — выдавать отрицательное число, то тогда, подставив предсказание линейной регрессии `z` в сигмоиду, мы сможем получать вероятности принадлежности к каждому из классов в диапазоне от `0` до `1`.

Далее по порогу вероятности мы сможем определять, к какому классу принадлежит объект.

Это и есть наша цель. Мы свели задачу классификации к задаче регрессии для предсказания вероятностей. 

Для бинарной классификации описанное выше будет выглядеть следующим образом:

<img src=ml3_img7.png>

<img src=ml3_img8.png>



## Разберёмся с геометрией

Возьмём частный случай, когда класс объекта зависит от двух признаков — `x1` и `x2`.

## Рассмотрим пример.

Мы пытаемся предсказать поступление студента в университет в зависимости от результатов двух экзаменов. Целевой признак  — результат поступления в аспирантуру (admission outcome) с двумя возможными значениями: поступил или не поступил. Факторы: `x1` — результат сдачи первого экзамена (Exam1 Score) и `x2` — результат сдачи второго (Exam 2 Score). Будем предсказывать вероятность поступления с помощью логистической регрессии.

Изобразим зависимость в пространстве двух факторов (вид сверху) в виде диаграммы рассеяния, а целевой признак отобразим в виде точек (непоступившие) и крестиков (поступившие).

Если рассматривать уравнение линейной регрессии отдельно от сигмоиды, то геометрически построить логистическую регрессию на основе двух факторов — значит найти такие коэффициенты `w0`, `w1` и `w2` уравнения плоскости, при которых наблюдается наилучшее разделение пространства на две части.

`z = w0 + w1*x1 + w2*x2`

Тогда выражение для `z` будет задавать в таком пространстве плоскость (в проекции вида сверху — прямую), которая разделяет всё пространство на две части. Над прямой вероятность поступления будет `>0.5`, а под прямой `<0.5`:

<img src=ml3_img9.png>

## В чём математический секрет?

Математически подстановка в уравнение плоскости точки, которая не принадлежит ей (находится ниже или выше), означает вычисление расстояния от этой точки до плоскости.

* Если точка находится ниже плоскости, расстояние будет отрицательным `z < 0`.
* Если точка находится выше плоскости, расстояние будет положительным `z > 0`.
* Если точка находится на самой плоскости, `z = 0`.

Мы знаем, что подстановка отрицательных чисел в сигмоиду приведёт к вероятности `P < 0.5`, а постановка положительных — к вероятности `P > 0.5`. 

>Таким образом, ключевым моментом в предсказании логистической регрессии является расстояние от точки до разделяющей плоскости в пространстве факторов. Это расстояние в литературе часто называется **отступом** (`margin`). 

В этом и состоит секрет работы логистической регрессии.

>Чем больше расстояние от точки, находящейся выше разделяющей плоскости, до самой плоскости, тем больше оценка вероятности принадлежности к классу 1.

Попробуйте подставить различные координаты точек в модель логистической регрессии и убедитесь в этом.

Можно построить тепловую карту, которая показывает, чему равны вероятности в каждой точке пространства:

<img src=ml3_img10.png>

>На рисунке точки, которые относятся к классу непоступивших абитуриентов, лежащие ниже разделяющей плоскости, находятся в красной зоне. Чем насыщеннее красный цвет, тем ниже вероятность того, что абитуриент поступит в аспирантуру. И наоборот, точки, которые относятся к классу поступивших абитуриентов, лежащие выше разделяющей плоскости, находятся в синей зоне. Чем насыщеннее синий цвет, тем выше вероятность того, что абитуриент поступит в аспирантуру.

<img src=ml3_img11.png>

<img src=ml3_img12.png>

# ПОИСК ПАРАМЕТРОВ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ <a class="anchor" id=2-2></a>

[к содержанию](#0)

Итак, мы разобрались с тем, как выглядит модель логистической регрессии и что она означает в геометрическом смысле.

>Но остался один главный вопрос: как найти такие коэффициенты `w = (w0, w1,... ,w_m)` , чтобы гиперплоскость разделяла пространство наилучшим образом?

Вновь обратимся к нашей схеме минимизации эмпирического риска:

<img src=ml3_img13.png width=800>

Можно предположить, что стоит использовать метод наименьших квадратов. Введём функцию ошибки — средний квадрат разности `MSE` между истинными классами `y` и предсказанными классами `y_` и попытаемся его минимизировать.

Сразу можно достоверно предсказать, что результат такого решения будет плохим, поэтому воздержимся от его использования.

Здесь нужен другой подход. Это **метод максимального правдоподобия** (`Maximum Likelihood Estimation — MLE`). 

>**Правдоподобие** — это оценка того, насколько вероятно получить истинное значение целевой переменной `y` при данных `x` и параметрах `w`. 

Данный метод позволяет получить функцию правдоподобия.

>Цель метода — найти такие параметры `w = (w0, w1,... ,w_m)`, в которых наблюдается максимум функции правдоподобия. Подробнее о выводе формулы вы можете прочитать здесь.

А мы пока что опустим математические детали метода и приведём только конечную формулу:

<img src=ml3_img14.png>
<img src=ml3_img15.png>

**Примечание**. К сожалению, функция `likelihood` не имеет интерпретации, то есть нельзя сказать, что значит число `2.34` в контексте правдоподобия.

## Цель — найти такие параметры, при которых наблюдается максимум этой функции.

Теперь пора снова применить магию математики, чтобы привести задачу к привычному нам формату минимизации эмпирического риска. По правилам оптимизации, если поставить перед функцией минус, то задача оптимизации меняется на противоположную: был поиск максимума — станет поиском минимума.

Таким образом мы получим функцию потерь `L(w)`, которая носит название «**функция логистических потерь**», или `logloss`. Также часто можно встретить название кросс-энтропия, или `cross-entropy` `loss`:

<img src=ml3_img16.png>

Математическую реализацию вычисления градиента для `logloss` мы обсудим далее в курсе, а пока нас интересует исключительно его смысл.

Мы уже знаем, что для того, чтобы повысить шанс пройти мимо локальных минимумов функции потерь, используется не сам градиентный спуск, а его модификации: например, можно использовать уже знакомый нам стохастический градиентный спуск (`SGD`).

Помним, что применение градиентного спуска требует предварительного масштабирования данных (стандартизации/нормализации). В реализации логистической регрессии в `sklearn` предусмотрено ещё несколько методов оптимизации, для которых масштабирование не обязательно. О них мы упомянём в практической части модуля.

Во избежание переобучения модели в функцию потерь логистической регрессии традиционно добавляется **регуляризация**. В реализации логистической регрессии в `sklearn` она немного отличается от той, что мы видели ранее для линейной регрессии.

<img src=ml3_img17.png>

# В этом месте можно полистать ноутбук `Classification.LogisticRegression.ipynb`

# 3. Метрики классификации. Мультиклассовая классификация <a class="anchor" id=3></a>

[к содержанию](#0)

# 4. Логическия регрессия. Практика <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Деревья решений <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Введение в ансамбли: бэггинг. Случайные лес <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Деревья решений и случайный лес. Практика <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)